In [ ]:
# This file is part of FAST-OAD_CS23-HE : A framework for rapid Overall Aircraft Design of Hybrid
# Electric Aircraft.
# Copyright (C) 2022 ISAE-SUPAERO

# Import pyVPLM modules/functions
from pyvplm.core.definition import PositiveParameter, PositiveParameterSet
from pyvplm.addon.variablepowerlaw import buckingham_theorem

# Declare physical variables and set (limits have no impact since DOE is not constructed)
t = PositiveParameter("t", [0.01, 46172.16929583475], "N", "trust")
RHO = PositiveParameter("RHO", [1.225], "kg/m^3", "air density")
n_rot = PositiveParameter("n_rot", [16.66, 33.33], "Hz", "rotational speed")
D = PositiveParameter("D", [2.31], "m", "propeller diameter")
pitch = PositiveParameter("pitch", [0.087, 1.0472], "rad", "propeller pitch")
PITCH_REF = PositiveParameter("PITCH_REF", [0.087], "rad", "propeller reference pitch")
v = PositiveParameter("v", [5.0, 191.37333333333333], "m/s", "air speed")
parameter_set = PositiveParameterSet(t, RHO, n_rot, D, pitch, PITCH_REF, v)

# Define d and lambda_wind to be first elected as repetitive set
parameter_set.first("n_rot", "D", "RHO")
parameter_set.latex_render()

# Calculate pi parameters with first repetitive set found in parameters' order
pi_set, _ = buckingham_theorem(parameter_set, False)
pi_set.latex_render()

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.core.definition import Constraint, ConstraintSet
from pyvplm.addon.pixdoe import create_const_doe
from pyvplm.addon.variablepowerlaw import (
    declare_func_x_to_pi,
    reduce_parameter_set,
    declare_constraints,
)
from pyvplm.addon.comsoladdon import save_file

reduced_parameter_set, reduced_pi_set = reduce_parameter_set(
    parameter_set, pi_set, elected_output="t"
)
reduced_pi_set.latex_render()
reduced_parameter_set.latex_render()

constraint1 = Constraint("n_rot*D/2<=300.0")
parameters_constraints = declare_constraints(reduced_parameter_set, ConstraintSet(constraint1))

# Save function to translate x parameters to pi parameters
func_x_to_pi = declare_func_x_to_pi(reduced_parameter_set, reduced_pi_set)

results = create_const_doe(
    reduced_parameter_set,
    reduced_pi_set,
    func_x_to_pi,
    whished_size=300,
    parameters_constraints=parameters_constraints,
)
doeX = results[0]

In [ ]:
import numpy as np

n_rot_val = np.array(doeX[:, 0])
D_val = np.array(doeX[:, 1])
RHO_val = np.array(doeX[:, 2])
pitch_val = np.array(doeX[:, 3])
PITCH_REF_val = np.array(doeX[:, 4])
v_val = np.array(doeX[:, 5])

aoa_75 = pitch_val - np.arctan2(v_val * 2, 0.75 * n_rot_val * D_val * 2.0 * np.pi)

proper_index = np.where(abs(aoa_75 - 0.05) < 0.22)[0]

new_doeX = doeX[proper_index, :]

# Save .txt file compatible with COMSOL
file_name = "./data/doe_fixed_prop"
save_file(new_doeX, file_name, reduced_parameter_set, is_SI=True)

n_rot_val = np.array(new_doeX[:, 0])
D_val = np.array(new_doeX[:, 1])
RHO_val = np.array(new_doeX[:, 2])
pitch_val = np.array(new_doeX[:, 3])
PITCH_REF_val = np.array(new_doeX[:, 4])
v_val = np.array(new_doeX[:, 5])

In [ ]:
import os.path as pth
from fastga_he.models.propulsion.components.propulsor.propeller.components.performance_point import (
    ComputePropellerPointPerformance,
)
from fastga.command.api import generate_block_analysis

# Define relative path
DATA_FOLDER_PATH = "data"
DATA_FILE = pth.join(DATA_FOLDER_PATH, "input_tbm900.xml")

# Define the wing primary geometry parameters name as a list
var_inputs = [
    "data:geometry:propeller:average_rpm",
    "data:geometry:propeller:diameter",
    "data:aerodynamics:propeller:point_performance:rho",
    "data:aerodynamics:propeller:point_performance:twist_75",
    "data:aerodynamics:propeller:point_performance:twist_75_ref",
    "data:aerodynamics:propeller:point_performance:speed",
]

# Declare function
compute_propeller = generate_block_analysis(
    ComputePropellerPointPerformance(),
    var_inputs,
    str(DATA_FILE),
    overwrite=False,
)

In [ ]:
thrust_array = np.zeros_like(n_rot_val)
power_array = np.zeros_like(n_rot_val)
eta_array = np.zeros_like(n_rot_val)

for idx, n_rot_it in enumerate(n_rot_val):
    inputs_dict = {
        "data:geometry:propeller:average_rpm": (n_rot_it, "rps"),
        "data:geometry:propeller:diameter": (D_val[idx], "m"),
        "data:aerodynamics:propeller:point_performance:rho": (RHO_val[idx], "kg/m**3"),
        "data:aerodynamics:propeller:point_performance:twist_75": (pitch_val[idx], "rad"),
        "data:aerodynamics:propeller:point_performance:twist_75_ref": (PITCH_REF_val[idx], "rad"),
        "data:aerodynamics:propeller:point_performance:speed": (v_val[idx], "m/s"),
    }
    output_dict = compute_propeller(inputs_dict)
    thrust = output_dict["data:aerodynamics:propeller:point_performance:thrust"][0][0]
    power = output_dict["data:aerodynamics:propeller:point_performance:power"][0][0]
    eta = output_dict["data:aerodynamics:propeller:point_performance:efficiency"][0][0]
    thrust_array[idx] = thrust
    power_array[idx] = power
    eta_array[idx] = eta

proper_result_idx = np.where(abs(eta_array - 0.5) < 0.5)[0]

In [ ]:
thrust_array = thrust_array[proper_result_idx]
power_array = power_array[proper_result_idx]
eta_array = eta_array[proper_result_idx]

n_rot_val = n_rot_val[proper_result_idx]
D_val = D_val[proper_result_idx]
RHO_val = RHO_val[proper_result_idx]
pitch_val = pitch_val[proper_result_idx]
PITCH_REF_val = PITCH_REF_val[proper_result_idx]
v_val = v_val[proper_result_idx]

In [ ]:
import pandas as pd

pi1 = thrust_array / (RHO_val * n_rot_val ** 2 * D_val ** 4)
pi2 = pitch_val / PITCH_REF_val
pi3 = v_val / (n_rot_val * D_val)
doeT = np.c_[pi1, pi2, pi3]
doeT = pd.DataFrame(doeT, columns=["pi1", "pi2", "pi3"])

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doeT.values, elected_pi0="pi1", order=2, log_space=False, ymax_axis=50, test_mode=True
)
perform_regression(doeT.values, model, choosen_model=4)

In [ ]:
pi1 = eta_array
doeP = np.c_[pi1, pi2, pi3]
doeP = pd.DataFrame(doeP, columns=["pi1", "pi2", "pi3"])

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doeP.values, elected_pi0="pi1", order=4, log_space=False, ymax_axis=50, test_mode=True
)
perform_regression(doeP.values, model, choosen_model=9)

# Trying eta as a function of Ct and J

In [ ]:
pi1 = eta_array
pi2 = thrust_array / (RHO_val * n_rot_val ** 2 * D_val ** 4)
pi3 = v_val / (n_rot_val * D_val)
doe_eta_best = np.c_[pi1, pi2, pi3]
doe_eta_best = pd.DataFrame(doe_eta_best, columns=["pi1", "pi2", "pi3"])

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doe_eta_best.values, elected_pi0="pi1", order=4, log_space=False, ymax_axis=50, test_mode=True
)
perform_regression(doe_eta_best.values, model, choosen_model=15)

print(pi1)

In [ ]:
import fastoad.api as oad
import fastga.utils.postprocessing.post_processing_api as api_plots

DATA_FILE_PROP = pth.join(DATA_FOLDER_PATH, "perf_prop_tbm900.xml")
reference_aircraft = oad.DataFile(DATA_FILE_PROP)

thrust = reference_aircraft["data:aerodynamics:propeller:sea_level:thrust"].value
speed_tbm900 = reference_aircraft["data:aerodynamics:propeller:sea_level:speed"].value
efficiency = reference_aircraft["data:aerodynamics:propeller:sea_level:efficiency"].value

fig = api_plots.propeller_efficiency_map_plot(DATA_FILE_PROP, sea_level=True)
fig.show()

In [ ]:
from scipy.interpolate import interp2d

effy_func = interp2d(thrust, speed_tbm900, efficiency)

n_tbm900 = reference_aircraft["data:geometry:propeller:average_rpm"].value[0] / 60
d_tbm900 = reference_aircraft["data:geometry:propeller:diameter"].value[0]
rho_tbm900 = 1.225

thrust_to_test = 10000.0
speed_to_test = 100.0

# j_mesh, ct_mesh = pi3, pi2
ct_mesh = thrust_to_test / (rho_tbm900 * n_tbm900 ** 2 * d_tbm900 ** 4.0)
j_mesh = speed_to_test / (n_tbm900 * d_tbm900)

effy = (
    -0.02812 * 2
    + 1.87164 * j_mesh
    - 2.120212 * j_mesh ** 2
    + 1.04901 * j_mesh ** 3
    - 0.25547 * j_mesh ** 4
    + 3.17893 * ct_mesh
    + 0.02305 * j_mesh ** 5
    + 2.49771 * ct_mesh * j_mesh
    - 25.39540 * ct_mesh ** 2
    - 19.13609 * ct_mesh ** 5
    - 1.530875 * ct_mesh ** 3 * j_mesh ** 2
    - 0.008 * ct_mesh * j_mesh ** 4
    + 52.21979 * ct_mesh ** 3
    - 9.04464 * ct_mesh ** 2 * j_mesh
    + 2.18030 * ct_mesh ** 2 * j_mesh ** 2
)
print(effy)
print(effy_func(thrust_to_test, speed_to_test))

In [ ]:
import plotly
import plotly.graph_objects as go

fig = go.Figure()
prop_scatter = go.Contour(
    x=pi3,
    y=pi2,
    z=effy,
    line_smoothing=0.85,
    ncontours=40,
    zmax=np.max(effy),
    zmin=0.0,
    colorscale="RdBu",
)
fig.add_trace(prop_scatter)
fig.show()

# Trying Cp as a function of Ct and J

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

pi1 = power_array / (RHO_val * n_rot_val ** 3 * D_val ** 5)
pi2 = thrust_array / (RHO_val * n_rot_val ** 2 * D_val ** 4)
pi3 = v_val / (n_rot_val * D_val)
doe_cp_best = np.c_[pi1, pi2, pi3]
doe_cp_best = pd.DataFrame(doe_cp_best, columns=["pi1", "pi2", "pi3"])

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doe_cp_best.values, elected_pi0="pi1", order=2, log_space=True, ymax_axis=10, test_mode=True
)
perform_regression(doe_cp_best.values, model, choosen_model=5)